## Importing Clean Data

In [2]:
#### Importing library
import pandas as pd
import numpy as np
import plotly.express as px
import nbformat
import plotly.graph_objects as go

#### Importing Clean file

In [3]:
orders = pd.read_csv("orders_clean.csv")
products = pd.read_csv("products_clean.csv")
reviews = pd.read_csv("reviews_clean.csv")
users = pd.read_csv("users_clean.csv")
products

,id,category,price,rating,title,vendor
0,1,Gizmo,29.4633,4.6,Rustic Paper Wallet,"Swaniawski, Casper and Hilll"
1,2,Doohickey,70.0799,0.0,Small Marble Shoes,Balistreri-Ankunding
2,3,Doohickey,35.3887,4.0,Synergistic Granite Chair,"Murray, Watsica and Wunsch"
3,4,Doohickey,73.9918,3.0,Enormous Aluminum Shirt,Regan Bradtke and Sons
4,5,Gadget,82.7451,4.0,Enormous Marble Wallet,"Price, Schultz and Daniel"
...,...,...,...,...,...,...
195,196,Widget,46.7641,0.0,Heavy-Duty Linen Toucan,Balistreri-Muller
196,197,Gizmo,46.7641,4.6,Aerodynamic Concrete Lamp,Erika Volkman Group
197,198,Gizmo,46.7641,4.1,Enormous Copper Shirt,"Considine, Schamberger and Schiller"
198,199,Widget,76.9533,3.6,Mediocre Leather Coat,"Gulgowski, Grimes and Mayer"


## Creating Product Profiling Table

In [4]:
# Editing initials product 

#Correcting Reviews Rating
# Compute average rating for each product
average_ratings = reviews.groupby('product_id')['rating'].mean().reset_index()
# Compute average review length
average_reviews_length = reviews.groupby('product_id')['review_length'].mean().reset_index()
# Compute amount of reviews per product
review_counts = reviews.groupby('product_id')['id'].count().reset_index()
review_counts.rename(columns={'id': 'review_count'}, inplace=True)
average_ratings.rename(columns={'rating': 'average_rating'}, inplace=True)


#Merging table
productstemp = products.merge(average_ratings, left_on="id", right_on="product_id", how= 'left')
productstemp2 = productstemp.merge(average_reviews_length, left_on="id", right_on="product_id", how= 'left')
productstemp2 = productstemp2.drop(["product_id_x","product_id_y"], axis= 1)
productstemp3 = productstemp2.merge(review_counts,left_on="id", right_on="product_id", how= 'left')
productstemp3 = productstemp3.drop(["product_id","rating"],axis=1)
productstemp3

product_profile = productstemp3.copy()
product_profile

,id,category,price,title,vendor,average_rating,review_length,review_count
0,1,Gizmo,29.4633,Rustic Paper Wallet,"Swaniawski, Casper and Hilll",4.625000,172.750000,8.0
1,2,Doohickey,70.0799,Small Marble Shoes,Balistreri-Ankunding,NaN,NaN,NaN
2,3,Doohickey,35.3887,Synergistic Granite Chair,"Murray, Watsica and Wunsch",4.000000,171.000000,7.0
3,4,Doohickey,73.9918,Enormous Aluminum Shirt,Regan Bradtke and Sons,3.000000,167.600000,5.0
4,5,Gadget,82.7451,Enormous Marble Wallet,"Price, Schultz and Daniel",4.000000,146.750000,4.0
...,...,...,...,...,...,...,...,...
195,196,Widget,46.7641,Heavy-Duty Linen Toucan,Balistreri-Muller,NaN,NaN,NaN
196,197,Gizmo,46.7641,Aerodynamic Concrete Lamp,Erika Volkman Group,4.666667,162.833333,6.0
197,198,Gizmo,46.7641,Enormous Copper Shirt,"Considine, Schamberger and Schiller",4.142857,197.000000,7.0
198,199,Widget,76.9533,Mediocre Leather Coat,"Gulgowski, Grimes and Mayer",3.666667,185.333333,6.0


## Creating Users Profile Table

In [5]:
product_profile.to_csv("product_profile.csv",index= False)